# Mask R-CNN Notebook for Cell Counting Model

## Prerequisites to running

ALWAYS run the following 2 cells. Make sure to check that you are on a GPU runtime. If you intend on using something else, make sure to change the second cell's DEVICE to reflect that.

In [ ]:
# Prereqs to running
# Always run this cell first

!pip install q keras==2.0.8
!pip install q keras==2.2.5
%tensorflow_version 1.x
!pip install q h5py==2.10.0
!rm -r sample_data/
!git clone https://github.com/Jotanator/115CellCount
%cd 115CellCount/MASK-R-CNN
!rm ../mask_rcnn_balloon_0030.h5

In [ ]:
# Always run this cell second

import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#from samples.balloon
import balloon

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "../logs")
#print(MODEL_DIR)

# Path to balloon data folder that contains train/ and val/
config = balloon.BalloonConfig()
BALLOON_DIR = os.path.join(ROOT_DIR, "data")
#print(BALLOON_DIR)

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

TEST_MODE = "inference"
def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Predicting

Run the below cells for running predictions on data in the val folder.
The images you want to predict on must be included in the via_region_data.json file to be valid. You will need to manually upload the .h5 file to 115CellCount. DO NOT run the cell that loads the model until the .h5 is finished uploading or else you might corrupt it and need to redownload it.

In [ ]:
# Load validation dataset
dataset = balloon.BalloonDataset()
dataset.load_balloon(BALLOON_DIR, "val")
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Change this to whatever file you want to load
weights_path = os.path.join(ROOT_DIR, "../mask_rcnn_balloon_0030.h5")
print(weights_path)

model.load_weights(weights_path, by_name=True)

In [ ]:
# Change this to any specific id you might want, or random choice
image_id = 0
#image_id = random.choice(dataset.image_ids)

image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

## Training

Run the below cell while current directory is inside samples/balloon. If it gives an error, try using a different keras version in the cell at the top of the notebook. You will need to restart your runtime and you will need to rerun the cells. Your files will all still be there. The models for each epoch will be put into a logs folder which will be at the same level as 115CellCount.

In [ ]:
#Train a new model starting from ImageNet weights
!python3 balloon.py train --dataset=data/ --weights=imagenet

#Train a new model starting from pre-trained COCO weights
#!python3 balloon.py train --dataset=data/ --weights=coco

#Resume training a model that you had trained earlier
#!python3 balloon.py train --dataset=data/ --weights=last